# Self attention

## Self attention without learning

In [166]:
# Definition of basic functions

# vector addition
def vectorAdd(v1, v2):
    if len(v1) != len(v2):
        raise ValueError("different vector lengths")
    return [v1[i] + v2[i] for i in range(len(v1))]


# vector addition for a list of vectors
def vectorAddList(vectorlist):
    v= vectorlist[0]
    for vi in vectorlist[1:]:
        v = vectorAdd(v, vi)
    return v

# scalar multiplication (vector)
def scalarMult(scalar, v):
    return [scalar * v[i] for i in range(len(v))]

# dot product
def dot(v1, v2):
    if len(v1) != len(v2):
        raise ValueError("different vector lengths")
    return sum(v1[i] * v2[i] for i in range(len(v1)))



# transpose
def t(M):
    return [[M[i][j] for i in range(len(M))] for j in range(len(M[0]))]

# matrix addition
def addMatrix(M1,M2):
    return [[M1[i][j] + M2[i][j] for j in range(len(M1[0]))] for i in range(len(M1))]


def multMatrix(M1,M2):
    # [[dot(ite Zeile von M1, jte Spalte von M2) for j in range(len(tM2))] for i in range(len(M1))]
    tM2 = t(M2)
    return [[dot(M1[i], tM2[j]) for j in range(len(tM2))] for i in range(len(M1))]

# listvector to matrix 
def v2m(v):
    return [v]

# 1-row matrix to vector: 
def m2v(v):
    return v[0]


def prettyprint(M):
    for i in range(len(M)):
        print(str(M[i])) 

In [146]:
a= [1,2,3] 
b= [1,0,1]
print("a + b =", vectorAdd(a, b), vectorAddList([a,b]))
print("2*a =", scalarMult(2, a))
print("a . b =", dot(a, b))



a + b = [2, 2, 4] [2, 2, 4]
2*a = [2, 4, 6]
a . b = 4


In [147]:
prettyprint(t([[1,2],[3,4],[5,6]]))

[1, 3, 5]
[2, 4, 6]


In [148]:
v2m([1,2,3])

[[1, 2, 3]]

In [149]:
M1 = [[1,2],[3,4],[5,6]]
M2 = [[-1,-2],[-3,-4],[5,6]]
prettyprint(M1)
print()
prettyprint(t(M2))
print()
prettyprint(multMatrix(M1,t(M2)))
print()
prettyprint(multMatrix(t(M1),M2))


[1, 2]
[3, 4]
[5, 6]

[-1, -3, 5]
[-2, -4, 6]

[-5, -11, 17]
[-11, -25, 39]
[-17, -39, 61]

[15, 16]
[16, 16]


Softmax Funktion $\sigma$:

$\sigma: \mathbf{R}^K \rightarrow (0,1)^K$ normalisierte Exponentialfunktion mit 

$ \sigma(\vec{x})_i = \frac{e^{x_{i}}}{\sum_{j=1}^K e^{x_{j}}}  \ for\ i=1,2,\dots,K  $



In [150]:
e = 2.718281828459

def softmax(x):
    return [e**x[i] / sum(e**x[j] for j in range(len(x))) for i in range(len(x))]

In [151]:
print(a,"softmax =", softmax(a))
print(b,"softmax =", softmax(b))
print([1,-10,-10],"softmax =", softmax([1,0,0]))
print([4],"softmax =", softmax([4]))



[1, 2, 3] softmax = [0.09003057317038284, 0.24472847105480003, 0.6652409557748171]
[1, 0, 1] softmax = [0.4223187982515171, 0.1553624034969658, 0.4223187982515171]
[1, -10, -10] softmax = [0.576116884765825, 0.21194155761708747, 0.21194155761708747]
[4] softmax = [1.0]


$\alpha$ ist die Relevanzmatrix, die sich aus dem Vergleich der Wortvektoren untereinander ergibt. Sei $L$ die Eingangstextlänge, dann ist

$\alpha_{ij} = softmax(score(x_i,x_j))\ \forall i,j \leq K$ die Relevanzmatrix.

Im Falle von *masked attention* vergleicht man den Wortvektor nur mit den vorangegangenen. Damit ergeben sich für ein Wort $x_i$ die Relevanzwerte:

$\alpha_{ij} = softmax(score(x_i,x_j))\ \forall j \leq i$ 

In [152]:
def score(x,y):
    return dot(x,y)

def alpha_masked(x): # x ist eine Liste von Wortvektoren
    alpha = []
    for i in range(len(x)):
        row = []
        for j in range(i+1):
            row.append(score(x[i], x[j]))
        alpha.append(softmax(row))
    return alpha

def alpha_masked_matrix(x): # x ist eine Liste von Wortvektoren
    alpha = []
    for i in range(len(x)):
        row = []
        for j in range(len(x)):
            if j <= i:
                row.append(score(x[i], x[j]))
            else:
                row.append(-float('inf'))  # use a large value to represent masked positions
        alpha.append(softmax(row))
    return alpha
    
def alpha(x): # x ist eine Liste von Wortvektoren
    alpha = []
    for i in range(len(x)):
        row = []
        for j in range(len(x)):
            row.append(score(x[i], x[j]))
        alpha.append(softmax(row))
    return alpha


In [153]:
vectorlist = [[2,4], [1,2], [0,2]]
alpha(vectorlist), alpha_masked(vectorlist)

([[0.9999484585145457, 4.539758978267296e-05, 6.143895671497805e-06],
  [0.9908674725821718, 0.006676412513377005, 0.002456114904451198],
  [0.9646631559719018, 0.017668422014049192, 0.017668422014049192]],
 [[1.0],
  [0.9933071490757145, 0.006692850924285412],
  [0.9646631559719018, 0.017668422014049192, 0.017668422014049192]])

Der kontextualisierte Output-Vektor  $y_i$ zu einem Inputvektor $x_i$ gegeben eine Liste von Wortvektoren $[x_j\ j\leq i]$ ist:

$$ y_i = \sum_{j\leq i} \alpha_{i,j} x_j  $$


In [154]:

def contextualized(x): # x ist eine Liste von Wortvektoren
    y = []
    for i in range(len(x)):
        a = alpha_masked(x)
        yi = vectorAddList([scalarMult(a[i][j] ,x[j]) for j in range(i+1)])
        y.append(yi)
    return y
 


In [155]:
contextualized([[1,2], [1,0], [1,1], [2,2]])

[[1.0, 2.0],
 [1.0, 1.0],
 [1.0, 1.5752103826044344],
 [1.8649548767993709, 1.9798697812543224]]

In [156]:
vectorlist = [[2, 4], [1, 2], [2, 0.1]]
vectorlist, contextualized(vectorlist)

([[2, 4], [1, 2], [2, 0.1]],
 [[2.0, 4.0],
  [1.9933071490757144, 3.9866142981514288],
  [1.938024701975659, 2.399131881320575]])

Liste von Wortvektoren: $[[2, 4], [1, 2], [2, 0.1]]$
Detaillierte Berechnung von $$ y_2 = \sum_{j\leq 2} \alpha_{2,j} x_j = \alpha_{2,0} x_0 + \alpha_{2,1} x_1 + \alpha_{2,2} x_2 $$ 
$$ [\alpha_{2,0},\alpha_{2,1},\alpha_{2,2}] = softmax([score(x_2,x_0), score(x_2,x_1), score(x_2,x_2)] )  = \\
softmax([4.4,2.2,4.01]) \approx [0.56,0.06,0.38] $$

**Vorsicht**: Der Vektor [2,0.1] hat ein höheres dot-Produkt mit dem Vektor [2,4] als mit sich selbst. Dies widerspricht der Idee, dass das dot-Produkt hier Ähnlichkeit modellieren soll. Ein Ausweg ist die explizite L2-Normalisierung der Wort-Embeddings:
Bevor das Dot-Produkt zur Berechnung von Relevanz-Scores verwendet wird, werden die Wortvektoren (Embeddings) explizit auf die Länge 1 normiert.

Statt einem Vektor $u$ wird der Vektor $\frac{u}{\|u\|}$ genutzt. Damit wird das dot-Produkt zur Cosinus-Ähnlichkeit: 
$$\text{cosine\_similarity}(u, v) = \frac{u \cdot v}{\|u\| \cdot \|v\|}$$

Hier werden wir dieses Problem erst einmal vernachlässigen.

In [164]:
w1 = "each"
wa2 = "session"
wb2 = "person"
w3 = "has"
w4 = "a"
w5 = "chair"



v1 = [4,3,3]
va2 = [7,2,1]
vb2 = [1,2,7]
v3 = [3.5,3,3.5]
v4 = [3,3,4]
v5 = [3,4,3]

s1 = [w1, wa2, w3, w4, w5]
s2 = [w1, wb2, w3, w4, w5]


embeddings = {w1:v1, wa2:va2, wb2:vb2, w3:v3, w4:v4, w5:v5}

print("Focus: embedding of '{}'".format(w5))

print("uncontextualized embedding of", "chair", "is", embeddings["chair"]) 
print("uncontextualized embedding of", "seesion", "is", embeddings["session"] ) 
print("uncontextualized embedding of", "person", "is", embeddings["person"]) 
print("\ncontextualized embedding of", "chair", "in", s1, "is")
print(contextualized([embeddings[w] for w in s1])[-1])
print("\ncontextualized embedding of", "chair", "in", s2, "is")
print(contextualized([embeddings[w] for w in s2])[-1])



Focus: embedding of 'chair'
uncontextualized embedding of chair is [3, 4, 3]
uncontextualized embedding of seesion is [7, 2, 1]
uncontextualized embedding of person is [1, 2, 7]

contextualized embedding of chair in ['each', 'session', 'has', 'a', 'chair'] is
[3.488241706560337, 3.3861879899594367, 3.1255703034802282]

contextualized embedding of chair in ['each', 'person', 'has', 'a', 'chair'] is
[3.1255703034802282, 3.3861879899594367, 3.4882417065603373]


## Self attention with learning

<img src="JM_F_9_10.png" width="500"> 

Aus Jurafsky & Martin (3rd edition, draft, January 12, 2025 release)

In [165]:
# Berechnung von projezierten Rollen-Vektoren
WQ = [[1,2,1],[-1,1,0],[-2,1,1]]
WK =  [[-1,-2,-1],[-1,-1,2],[0,-1,1]]
WV = [[-1,1,-2,2],[1,0,1,1],[1,0,0,-1]]

prettyprint(multMatrix(v2m(v5), WQ)) 
print()
prettyprint(multMatrix(v2m(v5), WK))
print()
prettyprint(multMatrix(v2m(v5), WV))

[-7, 13, 6]

[-7, -13, 8]

[4, 3, -2, 7]


<img src="JM_9_4.png" width="500"> 

Aus Jurafsky & Martin (3rd edition, draft, January 12, 2025 release)

<img src="JM_9_8
.png" width="500"> 


In [186]:
# Schrittweise Berechnung des 3. Outputs
print("Satz:",s1)
s_embedded = [embeddings[w] for w in s1]
print("Schritt 1: Embeddings", s_embedded)
rollen = [ {"k": multMatrix(v2m(v), WK), "q": multMatrix(v2m(v), WQ), "v": multMatrix(v2m(v), WV)} for v in s_embedded]
print("Schritt 2: projezierte Rollenvektoren", rollen)


Satz: ['each', 'session', 'has', 'a', 'chair']
Schritt 1: Embeddings [[4, 3, 3], [7, 2, 1], [3.5, 3, 3.5], [3, 3, 4], [3, 4, 3]]
Schritt 2: projezierte Rollenvektoren [{'k': [[-7, -14, 5]], 'q': [[-5, 14, 7]], 'v': [[2, 4, -5, 8]]}, {'k': [[-9, -17, -2]], 'q': [[3, 17, 8]], 'v': [[-4, 7, -12, 15]]}, {'k': [[-6.5, -13.5, 6.0]], 'q': [[-6.5, 13.5, 7.0]], 'v': [[3.0, 3.5, -4.0, 6.5]]}, {'k': [[-6, -13, 7]], 'q': [[-8, 13, 7]], 'v': [[4, 3, -3, 5]]}, {'k': [[-7, -13, 8]], 'q': [[-7, 13, 6]], 'v': [[4, 3, -2, 7]]}]


$$ score(x_i,x_j) = \frac{q_i\cdot q_j}{\sqrt{d_k}}$$

In [ ]:
# neue Sorefunktion mit erlernten Parametern
def scoreL(x1,x2):
    q1 = multMatrix(v2m(x1), WQ)
    k2 = multMatrix(v2m(x2), WK)
    return dot(m2v(q1),m2v(k2)) / len(x1)**0.5

In [188]:
rollen[2]["v"]

[[3.0, 3.5, -4.0, 6.5]]

In [191]:
# Input in softmax für 3. Wort

input = [scoreL(s_embedded[3],s_embedded[j]) for j in range(3)]
print("Schritt 3: Inputvektor für Softmax-layer", input )
print("Schritt 4: Softmax scores", softmax(input) )
weighted = [scalarMult(softmax(input)[i], m2v(rollen[i]["v"])) for i in range(3)]
print("Schritt 5: Weight each vector", weighted)
print("Schritt 6: Weighted sum", vectorAddList(weighted))


Schritt 3: Inputvektor für Softmax-layer [-91, -163, -81.5]
Schritt 4: Softmax scores [7.484622751062311e-05, 4.026866373829326e-36, 0.9999251537724895]
Schritt 5: Weight each vector [[0.00014969245502124623, 0.00029938491004249246, -0.0003742311375531156, 0.0005987698200849849], [-1.6107465495317303e-35, 2.818806461680528e-35, -4.832239648595191e-35, 6.040299560743989e-35], [2.9997754613174683, 3.4997380382037133, -3.999700615089958, 6.499513499521181]]
Schritt 6: Weighted sum [2.9999251537724896, 3.500037423113756, -4.000074846227511, 6.500112269341266]


In [194]:
# reshaping nedessary to get back to embedding dimension
WO = [[1,1,2],[3,1,2],[6,2,1],[0,0,1]]
print("Schritt 7: Reshaping to embedding dimension", multMatrix(v2m(vectorAddList(weighted)),WO))
print()
print("Contextualized embedding for 3rd word", multMatrix(v2m(vectorAddList(weighted)),WO))


Schritt 7: Reshaping to embedding dimension [[-10.500411654251309, -1.5001871155687763, 15.499962576886247]]

Contextualized embedding for 3rd word [[-10.500411654251309, -1.5001871155687763, 15.499962576886247]]
